# **TASK**

1. Dengan menggunakan data set adult buat perbandingan antar model dengan menggunakan CV.
2. Bandingkan model DT, KNN, LogReg, SVM.


nb: Coba untuk dilakukan hyper tuning untuk 2 model terbaik, bandingkan berdasarkan accuracy score

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler,StandardScaler,RobustScaler

from sklearn.model_selection import train_test_split, StratifiedKFold,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score, classification_report, precision_score,f1_score,recall_score

In [3]:
#Defining Function

def dataDescription(df):
    tempList = []
    for col in df.columns:
        tempList.append(
            [col,
            df[col].dtype,
            df[col].isna().sum(),
            round(df[col].isna().sum()/len(df)*100,2),
            df[col].nunique(),
            #list(df[col].drop_duplicates().sample(5,replace=True).values)
            list(df[col].drop_duplicates().sort_values().values)
            ]
        )

    descData = pd.DataFrame(data = tempList,
                            columns = ['Col','Data Type','Missing Value', 'Pct Missing Value','Num Unique','Unique Sample']
                            )
    display(descData)

## **Data Pre-Processing**

In [4]:
#Load Data
df = pd.read_csv('adult.csv')
display(df.info(),df.describe(),df.describe(exclude='number'),df.isnull().sum(),df.head(),dataDescription(df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


,Col,Data Type,Missing Value,Pct Missing Value,Num Unique,Unique Sample
0,age,int64,0,0.0,73,"[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2..."
1,workclass,object,0,0.0,9,"[?, Federal-gov, Local-gov, Never-worked, Priv..."
2,fnlwgt,int64,0,0.0,21648,"[12285, 13769, 14878, 18827, 19214, 19302, 193..."
3,education,object,0,0.0,16,"[10th, 11th, 12th, 1st-4th, 5th-6th, 7th-8th, ..."
4,education.num,int64,0,0.0,16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
5,marital.status,object,0,0.0,7,"[Divorced, Married-AF-spouse, Married-civ-spou..."
6,occupation,object,0,0.0,15,"[?, Adm-clerical, Armed-Forces, Craft-repair, ..."
7,relationship,object,0,0.0,6,"[Husband, Not-in-family, Other-relative, Own-c..."
8,race,object,0,0.0,5,"[Amer-Indian-Eskimo, Asian-Pac-Islander, Black..."
9,sex,object,0,0.0,2,"[Female, Male]"


None

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


,workclass,education,marital.status,occupation,relationship,race,sex,native.country,income
count,32561,32561,32561,32561,32561,32561,32561,32561,32561
unique,9,16,7,15,6,5,2,42,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,22696,10501,14976,4140,13193,27816,21790,29170,24720


age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


None

**SKEMA PRE-PROCESSING**
1. Missing Value : Fill Value 'NC'
2. One Hot Encoding : Relationship, Race, Sex
3. Binary Encoding: Workclass, Marital Status, Occupation, Native Country
4. Ordinal Encoding: Education (Already Encdoed)
5. No Treatment : Size
6. Take Out : fnlwgt

In [5]:
#Change '?' to missing value
df.replace('?',np.nan, inplace=True)

In [18]:
#model benchmarking
tree = DecisionTreeClassifier()
knn = KNeighborsClassifier()
logreg = LogisticRegression()
svm = SVC()

x = (tree,knn,logreg,svm)
x[1]

KNeighborsClassifier()

In [7]:
# Data Spliting

x = df.drop(columns=['fnlwgt','education','income'])
y = np.where(df['income']=='>50K',1,0)

xtrain, xtest, ytrain, ytest = train_test_split(
    x,
    y,
    stratify= y,
    random_state=100,
    test_size= 0.2
)

In [8]:
xtrain.isna().sum()

age                  0
workclass         1463
education.num        0
marital.status       0
occupation        1469
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     461
dtype: int64

In [9]:
#Skema Pre-Processing
BE_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant',fill_value='NC')),
    ('BE', ce.BinaryEncoder())
])

transformer = ColumnTransformer(
    [
        ('OHE', OneHotEncoder(drop='first'), ['relationship','race','sex']),
        ('Binary Enc', BE_pipeline,['workclass','marital.status','occupation','native.country'])
    ],
    remainder='passthrough' #LEWATKAN YANG GAK DIMENTION
)

crossVal_pipe = Pipeline([
    ('Preprocess', transformer),
    ('model',tree),
])
crossVal_pipe

transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('OHE', OneHotEncoder(drop='first'),
                                 ['relationship', 'race', 'sex']),
                                ('Binary Enc',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='NC',
                                                                strategy='constant')),
                                                 ('BE', BinaryEncoder())]),
                                 ['workclass', 'marital.status', 'occupation',
                                  'native.country'])])

In [10]:
# Transforming Data

# xtrain_prepros = transformer.fit_transform(xtrain)
# xtest_prepros = transformer.transform(xtest)
# xtrain_prepros

# xtrain_prepros = pd.DataFrame(xtrain_prepros)
# xtest_prepros = pd.DataFrame(xtest_prepros)
# xtrain_prepros

# feature = list(transformer.transformers_[0][1].get_feature_names_out()) + list(transformer.transformers_[1][1]['BE'].get_feature_names_out()) + ['age', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

# xtrain_prepros.columns = feature
# xtest_prepros.columns = feature

# xtrain_prepros.head()

perlu diperhatikan mean dan sandar reviasinya. kalau bisa di tampilkan juga n_splits nya

In [12]:
# Create Manual Cross Validation

# skfold = StratifiedKFold(n_splits=5)

# logreg_cv = cross_val_score(logreg,xtrain_prepros,ytrain, cv=skfold)
# tree_cv = cross_val_score(tree,xtrain_prepros,ytrain, cv=skfold)
# knn_cv = cross_val_score(knn,xtrain_prepros,ytrain, cv=skfold)
# svm_cv = cross_val_score(svm,xtrain_prepros,ytrain, cv=skfold)

In [13]:
#create looping to summarize data
from sklearn.svm import SVC
tree = DecisionTreeClassifier(max_depth = 10,criterion='entropy')
knn = KNeighborsClassifier(n_neighbors=10)
logreg = LogisticRegression()
svm = SVC(kernel = 'rbf')

model = [tree, knn, logreg, svm]
score = []
recall = []
std = []

skfold = StratifiedKFold(n_splits=5)

# i bisa diganti estimator = pipeline
for i in model:
    model_cv = cross_val_score(i,
                               xtrain,
                               ytrain,
                               cv=skfold,
                               scoring = 'recall' )
    score.append(model_cv)
    recall.append(model_cv.mean())
    std.append(model_cv.std())

result = pd.DataFrame({
    'Model' : ['Decision Tree','KNN','Logistic Regression','SVM'],
    'Recall': recall ,
    'Standard deviation' : std}
)

result.sort_values('Recall',ascending=False)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/akbarezamuhammad/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/akbarezamuhammad/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/Users/akbarezamuhammad/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 186, in fit
    X, y = self._validate_data(
  File "/Users/akbarezamuhammad/.pyenv/versions/3.9.1/lib/python3.9/site-packages/sklearn/base.py", line 579, in _validate_data
    X = check_array